<br>

# Introdução

In [ ]:
#!pip3 install traquitanas --upgrade

In [ ]:
import os
import time
import shutil
from datetime import date

In [ ]:
from traquitanas import net

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from paths import *

<br>

# Funções

<br>

## Cria Driver

In [ ]:
download_path = os.path.abspath(os.path.join(os.getcwd(), bruto_path, 'temp'))
print(download_path)

driver = net.scraping.create_driver(
    download_path,
    headless=False,
    adds_path=os.path.join('..', 'scrapy', 'adds'),
    log_path=os.path.join('..', 'scrapy', 'logs')
)

<br>

## *Download* dos dados

In [ ]:
# Vai pra site "Aplicações"
url = 'http://www.aplicacoes.daee.sp.gov.br/usosrec/fchweb.html'
driver.get(url)

In [ ]:
# Vai pra subsite "Baixar Usos p/Bacia Hidrográfica"
url = 'http://www.aplicacoes.daee.sp.gov.br/usosrec/Daeewebbacia1.asp'
driver.get(url)

In [ ]:
# Seleciona Bacia
ugrhi = Select(driver.find_element(by=By.XPATH, value="//*[@name='bacia1']"))
ugrhi.select_by_visible_text('99 - TODAS AS BACIAS HIDROGRAFICAS DO ESTADO DE SAO PAULO')

In [ ]:
# Clica em Selecionar (Esperar algum tepo)
driver.find_element(by=By.XPATH, value="//*[@value='SELECIONAR']").click()

In [ ]:
# Espera algum componente aparecer
time.sleep(5)

In [ ]:
# Fecha janela inicial
driver.switch_to.window(driver.window_handles[0])
driver.close()

# Muda para janela principal
driver.switch_to.window(driver.window_handles[0])

In [ ]:
WebDriverWait(driver, 80).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(@class, "caixa1")]')))

In [ ]:
# Scrool Down
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [ ]:
# Clicar para Download
driver.find_element(by=By.XPATH, value='//*[contains(@class, "caixa1")]').click()

In [ ]:
download_ok = False
n_tries = 0

while not download_ok and n_tries < 10:
    list_files = os.listdir(os.path.join(bruto_path, 'temp'))

    if len(list_files) > 1:
        time.sleep(1)

    if len(list_files) == 1:
        hoje = date.today().strftime('%Y.%m.%d')
        shutil.move(
            os.path.join(bruto_path, 'temp', list_files[0]),
            os.path.join(bruto_path, f'outorgas_{hoje}.txt'),
        )
    
    if len(list_files) == 0:
        # Espera download terminar
        driver.quit()
        time.sleep(1)
        os.chmod(os.path.join(bruto_path, 'temp'), 0o777)
        shutil.rmtree(
            os.path.join(bruto_path, 'temp'),
            onerror=print('ee'),
            ignore_errors=True
        )
        download_ok = True

    n_tries += 1